## COMP5328 - Advanced Machine Learning
## Assignment 1: Non-negative Matrix Factorization
----------------------------------------------------------------------------------------

**(Semester 2, 2025)**

In this ipython notebook, we provide some example code for assignment1.
+ Load Data.
    - ORL dataset. 
    - Extended YaleB dataset. 
    - AR dataset (**optional**).
+ Perform Evaluation. 
   - Relative Reconstruction Errors.
   - Accuracy, NMI (**optional**).

Lecturer: Tongliang Liu.

**Note: All datasets can be used only for this assignment and you are not allowed to distribute these datasets. If you want to use AR dataset, you need to apply it by yourself (we do not provide AR dataset due to the problem of license, please find more details in http://www2.ece.ohio-state.edu/~aleix/ARdatabase.html).**

## 1. Load Dataset

### 1.0 Data Folder

In [4]:
import os, warnings
os.environ["OMP_NUM_THREADS"] = "2"   # 避免 MKL+Windows 这个 known issue 的内存警告
warnings.filterwarnings(
    "ignore",
    message="KMeans is known to have a memory leak on Windows with MKL",
    category=UserWarning
)
#set file index
p =2

In [5]:
# The structure of data folder.
!ls -l data

total 0
drwxrwxrwx@ 41 eva-o2  staff  1312 Oct  7 21:48 CroppedYaleB
drwx------@ 44 eva-o2  staff  1408 Oct  7 21:35 ORL


### 1.1 Load ORL Dataset and Extended YaleB Dataset.
+ ORL dataset contains ten different images of each of 40 distinct subjects. For some subjects, the images were taken at different times, varying the lighting, facial expressions (open / closed eyes, smiling / not smiling) and facial details (glasses / no glasses). All the images were taken against a dark homogeneous background with the subjects in an upright, frontal position (with tolerance for some side movement). The size of each image is 92x112 pixels, with 256 grey levels per pixel. To further reduce the computation complexity, you can resize all images to 30x37 pixels.

+ Extended YaleB dataset contains 2414 images of 38 human subjects under 9 poses and 64 illumination conditions. All images are manually aligned, cropped, and then resized to 168x192 pixels. To further reduce the computation complexity, you can resize all images to 42x48 pixels.

In [7]:
import os
import numpy as np
from PIL import Image

#record the path to reload the origin data

def load_data(root='data/CroppedYaleB'):
    """ 
    Load ORL (or Extended YaleB) dataset to numpy array.
    
    Args:
        root: path to dataset.
        reduce: scale factor for zooming out images.
        
    """ 
    images, labels,paths = [], [],[]

    for i, person in enumerate(sorted(os.listdir(root))):
        
        if not os.path.isdir(os.path.join(root, person)):
            continue
        
        for fname in os.listdir(os.path.join(root, person)):    
            
            # Remove background images in Extended YaleB dataset.
            if fname.endswith('Ambient.pgm'):
                continue
            
            if not fname.endswith('.pgm'):
                continue
                
            # load image.
            img = Image.open(os.path.join(root, person, fname))
            #new -record the path
            img_path = os.path.join(root, person, fname)
            img = img.convert('L') # grey image.

            # TODO: preprocessing.
            # --- c fixed size (keeps PIL (W,H) order) ---
            root_name = os.path.basename(os.path.normpath(root)).lower()

            # --- normalize to [0,1] (nonnegative, NMF-safe) ---
            arr = np.asarray(img, dtype=np.float64) / 255.0
            # convert image to numpy column vector (after preprocessing).
            img = arr.reshape((-1, 1))

            #no-normalize
            img = np.asarray(img).reshape((-1,1))




            # collect data and label.
            images.append(img)
            labels.append(i)
            #collect the path
            paths.append(img_path)

    # concate all images and labels.
    images = np.concatenate(images, axis=1)
    labels = np.array(labels)

    return images, labels

## 2. 胡椒函数 +对应的噪音图展示 



In [9]:
import numpy as np
import matplotlib.pyplot as plt

#看起来正确的胡椒算法
def add_salt_pepper_noise(V, p= None, r=None, rng=None):
    V_noisy = V.copy()
    d, n = V.shape
    total_pixels = d * n
    num_corrupt = int(total_pixels * p)


    idx = rng.choice(total_pixels, size=num_corrupt, replace=False)
    num_white = int(num_corrupt * r)
    white_idx = idx[:num_white]
    black_idx = idx[num_white:]


    V_noisy.flat[white_idx] = 1.0
    V_noisy.flat[black_idx] = 0.0
    return V,V_noisy

# Plot result. 
def plot_noise_pic(V,img_size,ind = None, ps = None,rs = None):
    plt.figure(figsize=(10,4))
    plt.subplot(1, total, 1)
    plt.imshow(V_hat[:,ind].reshape(img_size[1],img_size[0]), cmap=plt.cm.gray)
    plt.title('Image(Original)')
    plt.subplot(1, total, 2)
    plt.imshow(V_noise[:,ind].reshape(img_size[1],img_size[0]), cmap=plt.cm.gray)
    plt.title('Noise')
    i = 3
    for p in ps:
        for r in rs:
            V_hat ,V  = add_salt_pepper_noise(V=V_hat,p=p,r=r )
            # Plot result. 
            plt.subplot(1, total, i)
            plt.imshow(V[:,ind].reshape(img_size[1],img_size[0]), cmap=plt.cm.gray,vmin= 0,vmax=1)
            plt.title("p={}, r={}".format(p, r))
            i+=1
    plt.tight_layout(pad=2.0)
    plt.show()
    






## 3. NMF代码实现 + NMF之后的可视化展示


In [11]:
#计算两个目标值之间的相对改进量
def relative_improvement(prev_obj, curr_obj, eps=1e-12):
    return abs(prev_obj - curr_obj) / (abs(prev_obj) + eps)

#定义统一的可视化函数
def visualize_reconstruction(V_hat, V_noisy, V_recon, ind=50, img_size=(92,112), title="NMF Reconstruction"):
    plt.suptitle(title, size=16)

    plt.figure(figsize=(12, 4))
    plt.subplot(131)
    plt.imshow(V_hat[:, ind].reshape(img_size[1],img_size[0]), cmap=plt.cm.gray)
    plt.title("Image (Original)")

    plt.subplot(132)
    plt.imshow(V_noisy[:, ind].reshape(img_size[1],img_size[0]), cmap=plt.cm.gray)
    plt.title("Image (Noisy)")

    plt.subplot(133)
    plt.imshow(V_recon[:, ind].reshape(img_size[1],img_size[0]), cmap=plt.cm.gray)
    plt.title("Image (Reconstructed)")

    plt.show()
    
eps = 1e-12

### 3.1 L1-NMF代码实现+实现后的效果可视化

In [13]:
import numpy as np

def L1_nmf(V, H, W, steps=3000, tol=1e-4, *,  # tol略放宽
           eps=1e-4,          # 稳定权重/分母，略放大
           omega_cap=1e3,     # 权重上限，避免过激
           alpha=0.5,         # 更新“降幂”=步长，0<alpha<=1
           floor=1e-12,       # 非负下限避免死列
           min_steps=100,     # 跑够一定步数再允许早停
           verbose=False):
    """Minimal but stabilized L1-NMF (IRLS + MU, few guards)."""
    V = np.maximum(np.asarray(V, float), floor)
    W = np.maximum(np.asarray(W, float), floor)
    H = np.maximum(np.asarray(H, float), floor)

    obj_prev = np.sum(np.abs(V - W @ H))

    for step in range(1, steps + 1):
        # IRLS权重：Ω = 1/(|E|+eps)，并裁顶
        E = V - (W @ H)
        Q = 1.0 / (np.abs(E) + eps)
        Q = np.minimum(Q, float(omega_cap))

        # --- 更新H（降幂 alpha 抑制过冲）---
        WH   = W @ H
        numH = W.T @ (Q * V)
        denH = W.T @ (Q * WH) + eps
        ratioH = numH / denH
        H = np.maximum(H * (ratioH ** alpha), floor)

        # --- 更新W（降幂 alpha）---
        WH   = W @ H
        numW = (Q * V) @ H.T
        denW = (Q * WH) @ H.T + eps
        ratioW = numW / denW
        W = np.maximum(W * (ratioW ** alpha), floor)

        # --- 列归一化W（稳尺度），尺度乘回H ---
        s = np.maximum(np.linalg.norm(W, axis=0), floor)
        W /= s
        H *= s[:, None]

        # --- 早停判据（相对改进）---
        obj = np.sum(np.abs(V - W @ H))
        rel = abs(obj_prev - obj) / (obj_prev + 1e-12)
        if verbose and step % 100 == 0:
            print(f"[L1-NMF] step={step:4d}  L1={obj:.6e}  rel={rel:.3e}")
        if (step >= min_steps) and (rel < tol):
            if verbose:
                print(f"[L1-NMF] Converged at step={step}, L1={obj:.6e}, rel={rel:.3e}")
            break
        obj_prev = obj

    return W, H, step

### 3.2 L2-NMF代码实现+实现后的效果可视化


For any data matrix $X$, L2-NMF finds non-negative factors $D \in \mathbb{R}^{w \times k}_{+}$ and $R \in \mathbb{R}^{k \times n}_{+}$ such that
$$
X \approx DR,
$$
by minimizing the Frobenius reconstruction error
$$
\min_{D,R \ge 0}\ \lVert X - DR \rVert_F^2 .
$$

Using multiplicative update rules (MUR), the iteration can be written as
$$
R \leftarrow R \odot \frac{D^\top X}{D^\top D R}, \qquad
D \leftarrow D \odot \frac{X R^\top}{D R R^\top},
$$
where $\odot$ denotes element-wise multiplication.



---------------------------


In [16]:
def L2_NMF(V,H,W,steps,tol=1e-5,verbose=False):
    # H is coeffient matrix
    # W is dictionary matrix
    # V is the matrix with noise
    #V hat  is the original matrix
    # print("basic_NMF")
    obj_prev = np.linalg.norm(V - W @ H, 'fro')**2
    
    for step in range(1, steps+1):    

        H_star = H * (W.T.dot(V)) / (W.T.dot(W).dot(H) + eps)
        W_star = W * (V.dot(H_star.T)) / (W.dot(H_star.dot(H_star.T)) + eps)

        H, W = H_star, W_star

        obj = np.linalg.norm(V - W @ H, 'fro')**2

        rel_impr = relative_improvement(obj_prev, obj, eps=eps)

        if rel_impr < tol:
            if verbose:
                print(f"Converged at step={step}, obj={obj:.6e}, rel_impr={rel_impr:.3e}")
            break

        obj_prev = obj

        if verbose and step in (1000, 5000, 10000, 15000, 20000):
            print(f"step={step}, obj={obj:.6e}, rel_impr={rel_impr:.3e}")

    return W,H, step


### 3.3 L2,1-NMF代码实现+实现后的效果可视化

In [18]:
def L21_nmf(V, H, W, steps=3000, tol=1e-5, verbose=False):

    def L21_obj(E):
        return np.sum(np.sqrt(np.sum(E * E, axis=1) + eps))

    obj_prev = L21_obj(V - W @ H)

    for step in range(1, steps + 1):
        # 残差 & 行权重
        E = V - (W @ H)                               # (d, n)
        row_norm = np.sqrt(np.sum(E * E, axis=1) + eps)  # (d,)
        s = 1.0 / (row_norm + eps)                    # (d,)

        # 广播实现 S = diag(s)
        SV  = s[:, None] * V
        SWH = s[:, None] * (W @ H)

        # H <- H .* (W^T (S V)) / (W^T (S W H))
        H *= (W.T @ SV) / (W.T @ SWH + eps)

        # W <- W .* ((S V) H^T) / ((S W H) H^T)
        SWH = s[:, None] * (W @ H)                    # 用更新后的 H 重新计算
        W  *= (SV @ H.T) / (SWH @ H.T + eps)

        # 收敛判定（相对改进）
        obj = L21_obj(V - W @ H)
        rel = relative_improvement(obj_prev, obj, eps=eps)
        # if verbose and (step % 100 == 0):
        #     print(f"[L2,1-NMF] step={step}, obj={obj:.6e}, rel={rel:.3e}")
        if rel < tol:
            if verbose:
                print(f"[L2,1-NMF] Converged at step={step}, obj={obj:.6e}, rel={rel:.3e}")
            break
        obj_prev = obj

    return W, H, step



### 3.4 CIM-NMF代码实现+实现后的效果可视化

For any data matrix $X$, CIM-NMF finds non-negative factors 
$D \in \mathbb{R}_+^{d \times k}$ and 
$R \in \mathbb{R}_+^{k \times n}$ such that

$$
X \approx DR,
$$

by minimizing the Correntropy-Induced Metric (CIM) reconstruction error:

$$
\min_{D,R \ge 0} \sum_{i=1}^d \sum_{j=1}^n 
\left( 1 - \frac{1}{\sqrt{2\pi}\delta} 
\exp\!\left(-\frac{(X_{ij} - (DR)_{ij})^2}{2\delta^2}\right) \right).
$$

---

### Multiplicative Update Rules (MUR)

Using the weight matrix

$$
W_{ij} = \frac{1}{\sqrt{2\pi}\delta} 
\exp\!\left(-\frac{(X_{ij} - (DR)_{ij})^2}{2\delta^2}\right),
$$

the iteration can be written as

$$
R \leftarrow R \odot \frac{D^\top (W \odot X)}{D^\top (W \odot (DR))}, 
\qquad
D \leftarrow D \odot \frac{(W \odot X) R^\top}{(W \odot (DR)) R^\top},
$$

where $\odot$ denotes element-wise multiplication.

In [20]:
def cim_nmf(V, H,W, max_iter=300, tol=1e-5, rng=None, verbose=False):
    """
    CIM-NMF (Correntropy-Induced Metric NMF)
    min_{W,H >=0} sum_{ij} r((V-WH)_{ij}, delta)

    更新规则基于权重加权的 NMF:
      H <- H * (W^T (R ⊙ V)) / (W^T (R ⊙ WH))
      W <- W * ((R ⊙ V) H^T) / ((R ⊙ WH) H^T)
    其中 R = exp(-(E^2)/(2 delta^2)) / (sqrt(2π) delta)
    """

    # m, n = V.shape
    # rng = np.random.default_rng(rng)
    # W = np.maximum(rng.random((m, k)), 1e-8)
    # H = np.maximum(rng.random((k, n)), 1e-8)

    obj_prev = np.inf
    eps = 1e-12
    delta = 1

    for it in range(1, max_iter + 1):
        # restucture
        V_hat = W @ H
        E = V - V_hat

        R = np.exp(-(E**2) / (2 * delta**2)) / (np.sqrt(2*np.pi) * delta)

        obj = np.sum(1 - R)

        # update H
        H *= (W.T @ (R * V)) / (W.T @ (R * V_hat) + eps)
        # update W
        V_hat = W @ H
        W *= ((R * V) @ H.T) / ((R * V_hat) @ H.T + eps)

        rel_impr = relative_improvement(obj_prev, obj, eps)
        if rel_impr < tol:
            if verbose:
                print(f"[CIM-NMF] Converged at iter={it}, obj={obj:.6e}, rel_impr={rel_impr:.3e}")
            break
        obj_prev = obj

        # if verbose and it % 50 == 0:
        #     print(f"[CIM-NMF] iter={it}, obj={obj:.6e}, rel_impr={rel_impr:.3e}")

    return W, H, it


## 4. Evaluation Metrics


### 4.1 Relative Reconstruction Errors (RRE)

To compare the robustness of different NMF algorithms, you can use the ```relative reconstruction errors```. Let $V$ denote the contaminated dataset (by adding noise), and $\hat{V}$
 denote the clean dataset. Let $W$ and $H$ denote the factorization results on $V$, the ``relative reconstruction errors`` then can be defined as follows:
 \begin{equation}
    RRE = \frac{ \| \hat{V} - WH \|_F }{ \| \hat{V} \|_F}.
\end{equation}


In [22]:
# # Evaluate relative reconstruction errors.
# 每个算法跑5次
def RRE_calcu(V_hat,V_star,algo,iter_time):
    # print(f"==> Evaluate RRE for {algo} for the {iter_time} time ")
    RRE = np.linalg.norm(V_hat - V_star) / np.linalg.norm(V_hat)
    # print('RRE = {}'.format(RRE))
    return RRE

### 4.2 Evaluate Clustering Performance

1. Accuracy.
    
    $$ Acc(Y, Y_{pred}) = \frac{1}{n}\sum\limits_{i=1}^n 1\{Y_{pred}(i) == Y(i)\}$$
        
2. Normalized Mutual Information (NMI).

    $$ NMI(Y, Y_{pred}) = \frac{2 * I(Y, Y_{pred})}{H(Y) + H(Y_{pred})} $$
    
   where $ I(\cdot,\cdot) $ is mutual information and $ H(\cdot) $ is entropy.

In [24]:
#other evaluate -检测指标
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import normalized_mutual_info_score

def assign_cluster_label(X, Y):
    kmeans = KMeans(n_clusters=len(set(Y))).fit(X)
    Y_pred = np.zeros(Y.shape)
    for i in set(kmeans.labels_):
        ind = kmeans.labels_ == i
        Y_pred[ind] = Counter(Y[ind]).most_common(1)[0][0] # assign label.
    return Y_pred


def accu_calcu(H,Y_hat,algo,iter_time):
    # print(f"==> Evaluate Acc and NMI for {algo} for the {iter_time} time ")
    # Assign cluster labels.
    Y_pred = assign_cluster_label(H.T, Y_hat)
    
    acc = accuracy_score(Y_hat, Y_pred)
    nmi = normalized_mutual_info_score(Y_hat, Y_pred)
    # print('Acc(NMI) = {:.4f} ({:.4f})'.format(acc, nmi))
    return acc, nmi



## 5. 执行鲁棒实验

In [26]:
#NMF algo_choice
# ===== L1 =====
def nmf_L1(V, K, local_seed, steps=3000, tol=1e-5, verbose=False):
    rng = np.random.RandomState(local_seed)
    N  = V.shape[1]   
    M  = V.shape[0]
    # init W, H
    W = rng.rand(M, K)
    H = rng.rand(K, N)
    W_star, H_star,step_star = L1_nmf(V, H, W, steps=steps, tol=tol, verbose=verbose)
    return W_star, H_star,step_star

# ===== L2=====
def nmf_L2(V, K, local_seed, steps=3000, tol=1e-5, verbose=False):
    rng = np.random.RandomState(local_seed)
    N  = V.shape[1]   
    M  = V.shape[0]
    # init W, H
    W = rng.rand(M, K)
    H = rng.rand(K, N)
    W_star, H_star,step_star = L2_NMF(V, H, W, steps=steps, tol=tol, verbose=verbose)
    return W_star, H_star,step_star

# ===== L2,1 =====
def nmf_L21(V, K, local_seed, steps=3000, tol=1e-5, verbose=True):
    rng = np.random.RandomState(local_seed)
    N  = V.shape[1]   
    M  = V.shape[0]
    # init W, H
    W = rng.rand(M, K)
    H = rng.rand(K, N)
    W_star, H_star,step_star = L21_nmf(V, H, W, steps=steps, tol=tol, verbose=True)
    return W_star, H_star,step_star

# ===== CIM =====
def nmf_CIM(V, K, local_seed, step=3000, tol=1e-5, verbose=True, **kwargs):
    rng = np.random.RandomState(local_seed)
    N  = V.shape[1]   
    M  = V.shape[0]
    # init W, H
    W = rng.rand(M, K)
    H = rng.rand(K, N)
    W_star, H_star,step_star = cim_nmf(V, H,W, max_iter=step, tol=tol, rng=rng, verbose=True, **kwargs)
    return W_star, H_star,step_star


In [27]:
def one_time_nmf(V_hat_all,Y_hat_all,algo,K,p,r,img_size,iter_time,local_seed):
    rng = np.random.RandomState(local_seed)
    
    #  90% subset
    n = V_hat_all.shape[1]
    m = int(np.ceil(0.9 * n))
    idx = rng.choice(n, size=m, replace=False)   
    V_sub = V_hat_all[:, idx]
    y_sub = Y_hat_all[idx]
    
    #add noise
    V_hat,V_noisy = add_salt_pepper_noise(V_sub, p=p, r=r, rng=rng)

    #algo
    W_star, H_star,step_star = algo(V_hat,K,local_seed)
    V_star = W_star @ H_star
    print(f"{algo} final run step:",step_star)
    # visualize_reconstruction(V_hat, V_noisy, V_star, 50, img_size, title=f"{algo} Reconstruction")

    #metrics-不确定该部分.应该用
    rre = RRE_calcu(V_hat,V_star,algo,iter_time)
    acc, nmi = accu_calcu(H_star,y_sub,algo,iter_time)
    
    return rre, acc, nmi 
    
def robost_pipeline(dataset, algos, noise_list,n_runs = 1,global_seed = 42):
    #load data
    V_hat, Y_hat = load_data(root=f"data/{dataset}")   
    print(f"{dataset} dataset: X.shape = {V_hat.shape}, Y.shape = {Y_hat.shape}")

    if dataset == "ORL":
        img_size = (92,112)
        K = 40
    elif dataset == "CroppedYaleB":
        img_size = (168,192)
        K= 38
    else: 
        print("can not find the dataset")

    #wolk through all the algo
    for algo_index, algo in enumerate(algos):
        #add_noise
        for noise_index,noise in enumerate(noise_list):
            p, r = noise
            rre_results = []
            acc_results = []
            nmi_results = []
            for i in range(n_runs):
                run_index = i
                local_seed = (algo_index+1)*100 + (noise_index+1)*10+run_index
                print(f"algo is {algo},noise is {noise}, iter_time is {run_index}, the local_seed is {local_seed}")

                rre, acc, nmi  = one_time_nmf(V_hat,Y_hat,algo,K,p,r,img_size,run_index,local_seed)
                rre_results.append(rre)
                acc_results.append(acc)
                nmi_results.append(nmi)
            mean_rre, std_rre = np.mean(rre_results), np.std(rre_results, ddof=1)
            mean_acc, std_acc = np.mean(acc_results), np.std(acc_results, ddof=1)
            mean_nmi, std_nmi = np.mean(nmi_results), np.std(nmi_results, ddof=1)

            print(f"[Algo={algo.__name__}, Noise={noise}, Run={run_index}] "
                  f"RRE={mean_rre:.4f}±{np.std(std_rre):.4f}, "
                  f"ACC={mean_acc:.4f}±{np.std(std_acc):.4f}, "
                  f"NMI={mean_nmi:.4f}±{np.std(std_nmi):.4f}")
                
    

## 6. 结果

In [ ]:
dataset_orl = 'ORL'
algos = [nmf_L1,nmf_L2,nmf_L21,nmf_CIM]
noise_list = [(0.2,0.1),(0.2,0.7),(0.4,0.1),(0.4,0.7),(0.6,0.1)]
r = 1

df_result = robost_pipeline(dataset_orl, algos, noise_list,n_runs = r,global_seed = 42)
dataset_yale = 'CroppedYaleB'
df_result_yale = robost_pipeline(dataset_yale, algos, noise_list,n_runs = r,global_seed = 42)

ORL dataset: X.shape = (10304, 400), Y.shape = (400,)
algo is <function nmf_L1 at 0x1588498a0>,noise is (0.2, 0.1), iter_time is 0, the local_seed is 110
<function nmf_L1 at 0x1588498a0> final run step: 3000
[Algo=nmf_L1, Noise=(0.2, 0.1), Run=0] RRE=0.1816±nan, ACC=0.7139±nan, NMI=0.8586±nan
algo is <function nmf_L1 at 0x1588498a0>,noise is (0.2, 0.7), iter_time is 0, the local_seed is 120


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


<function nmf_L1 at 0x1588498a0> final run step: 3000
[Algo=nmf_L1, Noise=(0.2, 0.7), Run=0] RRE=0.1819±nan, ACC=0.7667±nan, NMI=0.8732±nan
algo is <function nmf_L1 at 0x1588498a0>,noise is (0.4, 0.1), iter_time is 0, the local_seed is 130


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


<function nmf_L1 at 0x1588498a0> final run step: 3000
[Algo=nmf_L1, Noise=(0.4, 0.1), Run=0] RRE=0.1814±nan, ACC=0.6944±nan, NMI=0.8516±nan
algo is <function nmf_L1 at 0x1588498a0>,noise is (0.4, 0.7), iter_time is 0, the local_seed is 140


...

[CIM-NMF] Converged at iter=106, obj=2.235494e+06, rel_impr=9.949e-06
<function nmf_CIM at 0x158849c60> final run step: 106
[Algo=nmf_CIM, Noise=(0.4, 0.7), Run=0] RRE=0.1851±nan, ACC=0.6889±nan, NMI=0.8212±nan
algo is <function nmf_CIM at 0x158849c60>,noise is (0.6, 0.1), iter_time is 0, the local_seed is 450


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/yp/_gvtpr294n97pym7pydkrygm0000gn/T/ipykernel_24466/1556366779.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return abs(prev_obj - curr_obj) / (abs(prev_obj) + eps)


[CIM-NMF] Converged at iter=106, obj=2.235574e+06, rel_impr=9.986e-06
<function nmf_CIM at 0x158849c60> final run step: 106
[Algo=nmf_CIM, Noise=(0.6, 0.1), Run=0] RRE=0.1864±nan, ACC=0.7083±nan, NMI=0.8212±nan


/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


CroppedYaleB dataset: X.shape = (32256, 2414), Y.shape = (2414,)
algo is <function nmf_L1 at 0x1588498a0>,noise is (0.2, 0.1), iter_time is 0, the local_seed is 110


## 7. Visualization

In [ ]:
# Noise visualization
plot_noise_pic(V,img_size,ind = 50, ps = ps,rs = rs)
# # NMF -visualization
# visualize_reconstruction(V_hat, V_noisy, V_star, 50, img_size, title=f"{algo} Reconstruction")
    